## Introduction
标题已经太明显了好像没什么介绍可说了。

### 1.保存与重建 Variables
这一部分解释如何保存与重建变量(Estimators自动实现存储和重建变量)。

tf.train.Saver类提供存储和重建模型的方法，tf.train.Saver构建器为图中所有(或者指定列表中的)Variable添加保存和重建操作。Saver对象提供了运行这些操作的方法，指定了读取或者写入Checkpoints文件的路径。同样，Saver会重建所有已经定义于你的模型中的变量。

Tensorflow将Variable存储在二进制的Checkpoints文件中，简单来说，就是将Variable名字映射到tensor值。

#### 1.1 保存Variables
用tf.train.Saver()创建一个Saver来管理模型中的所有变量，例如，以下代码说明了如何调用tf.train.Saver.train方法并将Variable保存到Checkpoints文件中：

In [1]:
import tensorflow as tf

In [2]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
  # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    print("v1:",v1.eval())
    print("v2:",v2.eval())
  # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

v1: [1. 1. 1.]
v2: [-1. -1. -1. -1. -1.]
Model saved in path: /tmp/model.ckpt


#### 1.2 重建变量
注意当你重建变量之前不需要初始化

In [3]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
  # Check the values of the variables
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


#### Note: 并没有叫做“/tmp/model.ckpt”的物理文件，该名称只是所创建的Checkpoints文件前缀。

#### 1.3 选择保存和重建指定的Variable

如果你没有向tf.train.Saver()传入任何参数的话，Saver将处理该图中的所有Variable，每一个Variable在被创建时，根据其命名来保存。

有时指定Checkpoints文件中的Variable也是很有用的，例如，你也许训练了一个模型，有一个名为“weights”的Variable，在重建过程中，你想把该变量存储到另一个名为“params”的Variable中，或者说，有时你只想保存或者重建一个模型的子集，例如，你已经训练了一个五层的神经网络，现在你想训练一个六层的，并且重用已经存在的五层训练层，你可以保存并重用这仅有的五层网络参数。

你可以通过传递Variable名称给tf.train.Saver()来存储或者加载指定的Variable：
* 以list形式存储的Variable
* 以python字典形式存储的Variable：其中key为Variable的名称

In [4]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
  # Initialize v1 since the saver will not.
    v1.initializer.run()
    saver.restore(sess, "/tmp/model.ckpt")

    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


#### Note：注意v1需要初始化，也就是除了重建的变量，其余都需要初始化。

#### 1.4 检查Checkpoints文件中的Variable
我们可以通过inspect_library来快速查看某个Checkpoints文件中的Variable信息。

In [14]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='', all_tensors=True,all_tensor_names=True)

# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v1', all_tensors=False,all_tensor_names=False)

# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v2', all_tensors=False,all_tensor_names=False)

# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]
tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]


### 2.保存与重建模型概览
如果你想保存与加载Variable，Graph以及其他的元数据，基本来说，当你想保存或者重建你的模型的时候，我们推荐使用SavedModel。SavedModel使高层系统和工具可以使用，产生以及转化Tensorflow模型。Tensorflow提供几种与SavedModel互动的机制，包括tf.Saved_model APIs,Estimator APIS, 以及CLI.

### 3.构建及加载SavedModel的API们
这一章内容集中在如何调用API们来构建或者加载一个SavedModel，特别是在使用低层Tensorflow API时。
#### 3.1 构建一个SavedModel
我们提供SavedModel的python实现方法，SavedModelBuilder类提供了存储MetaGraphDef的函数。一个MetaFraph是一个数据流图，此外附带着variable，assets，以及signature。一个MetaGraphDef是一个协议缓存，代表一个MetaGraph。一个signature是一个Graph中输入与输出的集合。

如果这些asset，需要存储写入或者在硬盘上拷贝，当第一个MetaGraphDef被加入时，以上功能可用，如果多个MetaGraphDef都与同一个asset相关，只有第一个版本会被保留(说实话我没有看到哪里有asset的相关介绍)

每一个被加入到SavedModel中的MetaGraphDef必须用用户指定的标签来注释，这些标签提供加载或重建特定MetaGraphDef的方法，包括一些类共享的Variable以及assets。这些标签通常以MetaGraphDef的功能来命名(serving or training)，或者指定的硬件部分(GPU).

例如，以下代码给出使用SavedModelBuilder来创建SavedModel的典型方式（不好意思是伪代码）

In [ ]:
export_dir = ...
...
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:
  ...
  builder.add_meta_graph_and_variables(sess,
                                       [tag_constants.TRAINING],
                                       signature_def_map=foo_signatures,
                                       assets_collection=foo_assets)
...
# Add a second MetaGraphDef for inference.
with tf.Session(graph=tf.Graph()) as sess:
  ...
  builder.add_meta_graph([tag_constants.SERVING])
...
builder.save()

#### 3.2 用Python加载SavedModel
Python版本的SavedModel加载器提供兼容的加载以及重建SavedModel方法。load操作需要以下信息：
* Session：用于重建Model以及Variable
* Tags：用于指定加载的MetaGraphDef
* Location：SavedModel的目录

然后还是不能运行的伪代码

In [ ]:
"""
export_dir = ...
"""
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tag_constants.TRAINING], export_dir)
  ...

#### 用C++加载SavedModel这部分就略掉啦。。。

#### 用tensorflow Serving加载SavedModel这部分也略掉啦。。。

### Using SavedModel with Estimators
在训练一个Estimator模型后，你或许想从该模型创建可以接受请求并返回结果的服务，你可以在本地运行该服务，也可以部署在云中。

为了使用一个已训练良好的Estimator来提供服务，首先你必须将它导出为标准的SavedModel格式。这一章节解释了：
* 指定可以用于服务的相应节点和API们
* 将你的模型导出为SavedModel格式
* 在本地运行模型服务并处理预测请求